### `tes-thermo`
---

In this notebook, we will use `tes-thermo` to calculate the equilibrium compositions for the steam methane reforming (SMR) process.

Version 0.1.1 of `tes-thermo` allows the user to leverage the `thermo` library to retrieve information about the components and also enables the addition of new components.

First, import the Component and Gibbs classes. Then, specify the components to be considered.

In [2]:
%pip install -q tes-thermo==0.1.2

Note: you may need to restart the kernel to use updated packages.


In [3]:
from tes_thermo.utils import Component
from tes_thermo.gibbs import Gibbs
import numpy as np

new_components = {
        "methane": {
            "name": "methane",
            "Tc": 190.6, "Tc_unit": "K",
            "Pc": 45.99, "Pc_unit": "bar",
            "omega": 0.012,
            "Vc": 98.6, "Vc_unit": "cm³/mol",
            "Zc": 0.286,
            "Hfgm": -74520, "Hfgm_unit": "J/mol",
            "Gfgm": -50460, "Gfgm_unit": "J/mol",
            "structure": {"C": 1, "H": 4},
            "phase": "g",
            "kijs": [0, 0, 0, 0, 0, 0], # Interaction parameters with other components
            "cp_polynomial": lambda T: 8.314 * (1.702 + 0.009081* T -0.000002164*T**2),
        }
    }

components = ['water','carbon monoxide', 'carbon dioxide', 'hydrogen', 'methanol']

In `new_components`, the components to be added are defined. In this case, the user must specify all the thermodynamic properties of the component as well as the polynomial used to calculate $C_p$.

For this example, the following polynomial was used:

$$C_p(T) = R \times \left( 1.702 + 0.009081T - 0.000002164T^2 \right)$$

where $T$ is the temperature in Kelvin and $ C_p $ is the heat capacity in J/(mol·K).

The components listed in `components` will be instantiated using the `Chemical` class from the `thermo` library.

In [4]:
components = Component(components, new_components)
components = components.get_components()
gibbs = Gibbs(components=components,equation='Ideal Gas')
res = gibbs.solve_gibbs(T=1200, T_unit= 'K',
                         P=1.0, P_unit='bar',
                         initial=np.array([1, 0, 0, 0, 1, 0]))

2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Initializing Gibbs class...
2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Components created as Chemical from the thermo library: ['water', 'carbon monoxide', 'carbon dioxide', 'hydrogen', 'methanol']
2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Manually defined components: [{'name': 'methane', 'Tc': 190.6, 'Pc': 4599000.0, 'omega': 0.012, 'Vc': 9.859999999999998e-05, 'Zc': 0.286, 'Hfgm': -74520.0, 'Gfgm': -50460.0, 'phase': 'g', 'structure': {'C': 1, 'H': 4}, 'kijs': [0, 0, 0, 0, 0, 0], 'cp_polynomial': <function <lambda> at 0x103e25b20>}]
2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Equation of state: Ideal Gas
2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Starting Gibbs minimization problem resolution
2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Temperature: 1200.0 K, Pressure: 100000.0 Pa.
2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Phases identified - Solids: 0, Gases: 6
2025

[np.float64(-217718.73838899672), np.float64(-241808.25878361973), np.float64(-435817.8316102301), np.float64(-22724.499021260166), np.float64(-104515.72209531581), -14888.506381512696]


2025-07-26 18:31:23 - tes_thermo.gibbs.gibbs - INFO - Optimal solution found successfully


Note that the `Component` class takes both `components` and `new_components` as arguments. The `get_components` method already returns the required structure to instantiate the `Gibbs` class.

With this information, the `solve_gibbs` method returns the equilibrium compositions at temperature $T$ and pressure $P$, given the initial compositions specified by `initial`.

In [5]:
res

{'Temperature (K)': 1200.0,
 'Pressure (bar)': 1.0,
 'Water': 0.7639475494006989,
 'Carbon monoxide': 0.7633496223323687,
 'Carbon dioxide': 0.23635142729379255,
 'Hydrogen': 2.2354545448851115,
 'Methanol': 3.6241292561174665e-09,
 'Methane': 0.0002989665348374862}